In [1]:
# Loading packages
using CSV, DataFrames, Gurobi, JuMP

In [2]:
# input data
order_list = CSV.read("input/order_list.csv", DataFrame)
freight_rates = CSV.read("input/freight_rates.csv", DataFrame)
wh_costs = CSV.read("input/wh_costs.csv", DataFrame)
wh_capacities = CSV.read("input/wh_capacities.csv", DataFrame)
products_per_plant = CSV.read("input/products_per_plant.csv", DataFrame)
vmi_customers = CSV.read("input/vmi_customers.csv", DataFrame)
plant_ports = CSV.read("input/plant_ports.csv", DataFrame);

In [3]:
order_list = order_list[1:1000, :]

Row,Order ID,Origin Port,Carrier,Service Level,Customer,Product ID,Plant Code,Destination Port,Unit quantity,Weight,Maximum delivery time
,Int64,Int64,Float64?,Int64,Int64,Int64,Int64,Int64,Int64,Float64,Int64
1,5276,9,missing,0,37,1481,16,1,808,14.3,4
2,691,9,missing,0,37,1481,16,1,3188,87.94,4
3,68,9,missing,0,37,1481,16,1,2331,61.2,4
4,7361,9,missing,0,37,1481,16,1,847,16.16,4
5,7363,9,missing,0,37,1481,16,1,2163,52.34,4
6,7049,9,missing,0,37,1481,16,1,3332,92.8,4
7,6059,9,missing,0,37,1481,16,1,1782,46.9,4
8,692,9,missing,0,37,1481,16,1,427,2.86,4
9,3041,9,missing,0,37,1481,16,1,1291,26.6,4


In [4]:
# Extract information from input data
## numbers
norder = 1000 # number of all the orders
nwarehouse = 19 # number of all the warehouses
nfreight = 1540 # number of all the freight choices
nproduct = 1540 # number of all the products
ncustomer = 48 # number of all the customers
nwport = 11 # number of all the warehouse ports
# Well done!!!

## data
### from order list sheet
s = Vector(order_list[!, "Service Level"]) # service level s[k]
q = Vector(order_list[!, "Unit quantity"]) # product quantity q[k] for order k
pro = Vector(order_list[!, "Product ID"]) # product id for order k
cus = Vector(order_list[!, "Customer"]) # customer id for order k
ot = Vector(order_list[!, "Maximum delivery time"]) # maximum delivery time for order k
ow = Vector(order_list[!, "Weight"]) # order weight for order k

### from freight rates sheet
o = Vector(freight_rates[!, "orig_port_cd"]) # origin port o[j]
d = Vector(freight_rates[!, "dest_port_cd"]) # destination port d[j]
c = Vector(freight_rates[!, "Carrier"]) # Carrier c[j]
a = Vector(freight_rates[!, "Area"]) # storage area number a[j]
t = Vector(freight_rates[!, "tpt_day_cnt"]) # delivery time p[j]
m = Vector(freight_rates[!, "mode_dsc"]) # transportation mode m[j]
r = Vector(freight_rates[!, "rate"]) # transportation rate cost per unit product r[j]
minc = Vector(freight_rates[!, "minimum cost"]); # minimum transportation rate cost per unit product r[j]
minw = Vector(freight_rates[!, "minm_wgh_qty"]); # minimum transportation weight
maxw = Vector(freight_rates[!, "max_wgh_qty"]); # maximum transportation weight

## from warehouse capacities sheet
cap = Vector(wh_capacities[!, "Daily Capacity"])

## from warehouse costs sheet
p = Vector(wh_costs[!, "Cost/unit"]) # warehouse p[i]

## Big-M
M = 1;

In [5]:
# create PW_{ui}
## create a list for the products per warehouse
products_per_warehouse_list = []
for i in 1:size(products_per_plant, 1)
    push!(products_per_warehouse_list, (products_per_plant[i, 1], products_per_plant[i, 2]))
end

## create a matrix for the products per plant
PW = zeros(Int, nproduct, nwarehouse)
for (i, u) in products_per_warehouse_list
    PW[u, i] = 1
end

# create WP_{ie}
## create a list for the warehouse port per warehouse
warehouse_port_per_warehouse_list = []
for i in 1:size(plant_ports, 1)
    push!(warehouse_port_per_warehouse_list, (plant_ports[i, 1], plant_ports[i, 2]))
end

## create a matrix for the warehouse port per warehouse
WP = zeros(Int, nwarehouse, nwport)
for (i, e) in warehouse_port_per_warehouse_list
    WP[i, e] = 1
end

# create WC_{iv}
## create a list for the customers per warehouse
customers_per_warehouse_list = []
for i in 1:size(vmi_customers, 1)
    push!(customers_per_warehouse_list, (vmi_customers[i, 1], vmi_customers[i, 2]))
end

## create a matrix for the customers per warehouse
WC = zeros(Int, nwarehouse, ncustomer)
for (i, v) in customers_per_warehouse_list
    WC[i, v] = 1
end

In [6]:
o

1540-element Vector{Int64}:
 8
 8
 8
 8
 8
 8
 8
 8
 8
 8
 8
 8
 8
 ⋮
 4
 4
 2
 2
 2
 2
 2
 3
 3
 3
 3
 3

In [7]:
# Build the model
# create a model
model = Model(Gurobi.Optimizer)

Set parameter Username
Academic license - for non-commercial use only - expires 2024-08-17


A JuMP Model
Feasibility problem with:
Variables: 0
Model mode: AUTOMATIC
CachingOptimizer state: EMPTY_OPTIMIZER
Solver name: Gurobi

In [8]:
# define decision variables
@variable(model, X[1:norder, 1:nwarehouse], Bin) # X[k,i] = 1 if order k is assigned to warehouse i
@variable(model, Y[1:norder, 1:nfreight], Bin) # Y[k,f] = 1 if order k is assigned to freight f

# define other variables
@variable(model, TC[1:norder, 1:nfreight]>=0) # Transportation cost TC[k, j] of order k assigned to freight j
@variable(model, TCA[1:norder, 1:nfreight]>=0) # Air transportation cost TCA[k, j] of order k assigned to freight j
@variable(model, TCG[1:nfreight]>=0) # Ground transportation cost TCG[j] for all routes j
@variable(model, z[1:nfreight]>=0);

$$
\text{Objective} = \min (\text{Warehouse Cost} + \text{Transportation Cost}) \\
\min \sum_{k=1}^{n_{\text{order}}}\sum_{i=1}^{n_{\text{warehouse}}} X_{ki} \cdot p_{i} \cdot q_{k} + \sum_{k=1}^{n_{\text{order}}}\sum_{j=1}^{n_{\text{freight}}} Y_{kj} \cdot TC_{kj}
$$

In [9]:
# define the objective function
@objective(model, Min, sum(X[k, i] * p[i] * q[k] for i in 1:nwarehouse for k in 1:norder) + sum(Y[k, j] * TC[k, j] for k in 1:norder for j in 1:nfreight));

- each order needs to be assigned to a warehouse
$$
\sum_{i=1}^{n_{\text{warehouse}}} X_{ki} = 1, \forall k
$$

In [10]:
@constraint(model, warehouse_allocation_constraint[k in 1:norder], sum(X[k,i] for i in 1:nwarehouse) == 1);

- each order needs to be assigned to a freight assignment
$$
\sum_{j=1}^{n_{\text{freight}}} Y_{kj} = 1, \forall k
$$

In [11]:
@constraint(model, freight_constraint[k in 1:norder], sum(Y[k,j] for j in 1:nfreight) == 1);

- each warehouse has a daily order capacity 
$$
\sum_{k=1}^{n_{\text{order}}} X_{ki} \leq cap_{i}, \forall i
$$

In [12]:
@constraint(model, order_capacity_constraint[i in 1:nwarehouse], sum(X[k,i] for k in 1:norder) <= cap[i]);

- each product can be stored in some warehouses only
$$
X_{ki} \leq M(PW_{pro_{k}i}), \forall k,i
$$

In [13]:
@constraint(model, warehouse_prod_constraint[k in 1:norder, i in 1:nwarehouse], X[k,i] <= M*PW[pro[k],i]);

- some warehouses can only service certain customers
$$
X_{ki} \leq M(WC_{icus_{k}}), \forall k,i
$$

In [14]:
# @constraint(model, warehouse_cus_constraint[k in 1:norder, i in 1:nwarehouse], X[k,i] <= M*WC[i,cus[k]]);

- each warehouse can only begin transporting things via some specific warehouse ports
$$
Y_{kj} \leq M(1-X_{ki}+WP_{io_{j}}), \forall k,i,j
$$

In [15]:
@constraint(model, warehouse_wport_constraint[k in 1:norder, i in 1:nwarehouse, j in 1:nfreight], Y[k,j] <= M*(1-X[k,i]+WP[i,o[j]]));

- orders need to be shipped within a certain time to the customer
$$
\sum_{j=1}^{n_{\text{freight}}} Y_{kj} \cdot t_{j} \leq ot_{k}, \forall k
$$

In [16]:
# @constraint(model, shipping_constraint[k in 1:norder], sum(Y[k,j]*t[j] for j in 1:nfreight) <= ot[k]);

- for each route via a carrier, different parts of the carrier should meet a maximum weight
$$
\sum_{k=1}^{n_{\text{order}}} Y_{kj} \cdot ow_{k} \leq maxw_{j}, \forall j
$$

In [17]:
@constraint(model, weight_constraint[j in 1:nfreight], sum(Y[k,j]*ow[k] for k in 1:norder) <= maxw[j]);

- total transportation cost
$$
TC_{kj} \leq s_{k}[(1-m_{j}) \cdot TCA_{kj} + m_{j} \cdot TCG_{j}], \forall k, j
$$

In [18]:
@constraint(model, transportation_cost_constraint[k in 1:norder, j in 1:nfreight], TC[k,j] <= s[k]*((1-m[j])*TCA[k,j] + m[j]*TCG[j]));

- air transportation cost 
$$
minc_{j} \leq TCA_{kj}, \forall k, j
$$
$$
ow_{k} \cdot r_{j} \leq TCA_{kj}, \forall k, j
$$

In [19]:
@constraint(model, air_mincost_constraint[k in 1:norder, j in 1:nfreight], TCA[k,j] >= minc[j]);
@constraint(model, air_minweight_constraint[k in 1:norder, j in 1:nfreight], TCA[k,j] >= ow[k]*r[j]);

- ground transportation cost
$$
z_{j}r_{j} \leq TCG_{j}, \forall j
$$
$$
z_{j} \leq \sum_{k=1}^{n_{\text{order}}} Y_{kj}, \forall j
$$
$$
Y_{kj} \leq z_{j}, \forall k, j
$$

In [20]:
@constraint(model, ground_cost_constraint1[j in 1:nfreight], z[j]*r[j] <= TCG[j]);
@constraint(model, ground_cost_constraint2[j in 1:nfreight], z[j] <= sum(Y[k,j] for k in 1:norder));
@constraint(model, ground_cost_constraint3[k in 1:norder, j in 1:nfreight], z[j] >= Y[k,j]);

In [ ]:
# solve the model
optimize!(model)

In [ ]:
# # Retrieve the objective value
objective = JuMP.objective_value(model)

In [ ]:
# # Retrieve the solution
solution_X = JuMP.value.(X)
solution_Y = JuMP.value.(Y)

In [ ]:
sum_warehouse = vec(sum(solution_X, dims=1))

In [ ]:
sum_freight = vec(sum(solution_Y, dims=1))

In [ ]:
CSV.write("output/Y.csv", DataFrame(Vector=sum_freight))